In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19056528/tmplvsj9vha
  JVM stdout: /state/partition1/job-19056528/tmplvsj9vha/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19056528/tmplvsj9vha/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,"21 days, 2 hours and 10 minutes"
H2O_cluster_name:,H2O_from_python_mk7516_l2vjf5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(531)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(1296, 7)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220504_174225_model_170


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,359.0,1436.0,266599.0,3.0,6.0,4.967967,4.0,12.0,10.053621




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 4.407601914994625e-05
RMSE: 0.006638977266864698
LogLoss: 0.0022380394213852607
Mean Per-Class Error: 0.0
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,Error,Rate
0,916.0,0.0,0.0,0.0,0.0,0 / 916
1,0.0,281.0,0.0,0.0,0.0,0 / 281
2,0.0,0.0,45.0,0.0,0.0,0 / 45
3,0.0,0.0,0.0,54.0,0.0,0 / 54
4,916.0,281.0,45.0,54.0,0.0,"0 / 1,296"



Top-4 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.995370,0.004230,0.996154,0.988417,0.996139,1.000000,0.996139
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.004630,0.004230,0.003846,0.011583,0.003861,0.000000,0.003861
3,err_count,1.200000,1.095445,1.000000,3.000000,1.000000,0.000000,1.000000
4,logloss,0.034518,0.019656,0.045172,0.061000,0.013308,0.017780,0.035330
5,max_per_class_error,0.025611,0.029625,0.017544,0.016043,0.076923,0.000000,0.017544
6,mean_per_class_accuracy,0.993597,0.007406,0.995614,0.995989,0.980769,1.000000,0.995614
7,mean_per_class_error,0.006403,0.007406,0.004386,0.004011,0.019231,0.000000,0.004386
8,mse,0.007369,0.005555,0.008978,0.016164,0.002944,0.002644,0.006116
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-04 18:20:58,9 min 30.835 sec,0.0,0.750000,1.386294,0.444444,NaN,NaN
1,,2022-05-04 18:20:58,9 min 30.845 sec,5.0,0.528577,0.760700,0.148920,NaN,NaN
2,,2022-05-04 18:20:58,9 min 30.856 sec,10.0,0.417637,0.526517,0.124228,NaN,NaN
3,,2022-05-04 18:20:58,9 min 30.867 sec,15.0,0.358925,0.401871,0.118827,NaN,NaN
4,,2022-05-04 18:20:58,9 min 30.880 sec,20.0,0.323491,0.328086,0.094907,NaN,NaN
5,,2022-05-04 18:20:58,9 min 30.891 sec,25.0,0.293717,0.273870,0.076389,NaN,NaN
6,,2022-05-04 18:20:58,9 min 30.904 sec,30.0,0.268475,0.234205,0.060185,NaN,NaN
7,,2022-05-04 18:20:58,9 min 30.916 sec,35.0,0.247958,0.205239,0.047840,NaN,NaN
8,,2022-05-04 18:20:58,9 min 30.928 sec,40.0,0.229020,0.181015,0.042438,NaN,NaN
9,,2022-05-04 18:20:58,9 min 30.941 sec,45.0,0.212827,0.161675,0.033179,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,safety,677.198181,1.000000,0.352564
1,persons,573.663574,0.847113,0.298662
2,buying,295.359009,0.436149,0.153770
3,maint,233.295502,0.344501,0.121459
4,lug_boot,98.133759,0.144911,0.051091
5,doors,43.131290,0.063691,0.022455


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_1_AutoML_1_20220504_174225_model_170,0.00722777,0.0344697,0.0858339,0.00736746
GBM_grid_1_AutoML_1_20220504_174225_model_228,0.00846127,0.0190143,0.0640352,0.00410051
GBM_grid_1_AutoML_1_20220504_174225_model_137,0.0127833,0.0265104,0.0747277,0.00558422
GBM_grid_1_AutoML_1_20220504_174225_model_32,0.0131272,0.0239912,0.0717142,0.00514293
GBM_grid_1_AutoML_1_20220504_174225_model_269,0.0134001,0.0255703,0.0720504,0.00519127
GBM_grid_1_AutoML_1_20220504_174225_model_127,0.0136368,0.0315694,0.0837526,0.00701451
GBM_grid_1_AutoML_1_20220504_174225_model_107,0.0139097,0.0252261,0.0802805,0.00644495
GBM_grid_1_AutoML_1_20220504_174225_model_177,0.0139459,0.0317016,0.0828102,0.00685753
GBM_grid_1_AutoML_1_20220504_174225_model_285,0.0139459,0.0276326,0.0790087,0.00624238
GBM_grid_1_AutoML_1_20220504_174225_model_241,0.0139459,0.0266208,0.0773566,0.00598404
